使用tensorflow开源的Cifar10来对数据进行读取
这里，我把它的代码搬到了我的笔记的同级目录下面了，下载google的代码仓库可以执行下面的命令
```shell
    git clone https://github.com/tensorflow/modles.git
```
然后进入cifar10这个项目的目录中就可以看见它的源代码了。
```shell
    cd models/tutorials/image/cifar10
```

下面是这个神经网络的流程结构图：

|Layer名称|描述|
|---------|----|
|conv1|卷积层和ReLU激活函数|
|pool1|最大池化|
|norm1|LRN|
|conv2|卷积层和ReLU激活函数|
|norm2|LRN|
|pool2|最大池化|
|fla1|全连接层和ReLU激活函数|
|fla2|全连接层和ReLU激活函数|
|logits|模型Inference输出结果|
+ **input_image**
 - 数据增强
 
+ **conv1**
 - 卷积层（权重不使用L2正则化）卷积核[5,5,3,64]，步长[1,1,1,1],Padding="SAME"
 - 激活函数ReLU
 - 最大池化层ksize[1,3,3,1], 步长[1,2,2,1], Padding="SAME"
 - LRN算法

+ **conv2**
 - 卷积层（权重不使用L2正则化）卷积核[5,5,64,64]，步长[1,1,1,1],Padding="SAME"
 - 激活函数ReLU
 - LRN算法
 - 最大池化层ksize[1,3,3,1], 步长[1,2,2,1], Padding="SAME"
 
+ **fla1**
 - 权重使用L2正则化
 - 节点数 384
 - reshape:[batch_size, -1]
 - 激活函数ReLU

+ **fla2**
 - 权重使用L2正则化
 - 节点数 192
 - 激活函数ReLU

+ **fla3**
 - 权重不使用L2正则化
 - 节点数 10
 - 不使用激活函数

In [1]:
import cifar10, cifar10_input
import tensorflow as tf
import numpy as np
import time

这里我手动下载了cifar-10的数据，并把它解压在了我的D盘的download文件夹下面的cifar-10-data目录下面了，因此我这里写的data_dir = "D:\download\cifar-10-data\cifar-10-batches-bin",数据的下载地址为：[http://www.cs.toronto.edu/~kriz/cifar-10-binary.tar.gz](http://www.cs.toronto.edu/~kriz/cifar-10-binary.tar.gz)


这里设置总共迭代数据3000次

每一个mini-batch-size为128

In [2]:
max_step = 3000
batch_size = 128
data_dir = "D:\download\cifar-10-data\cifar-10-batches-bin"

定义初始化权重的函数，和之前不同的是，这里 给weight添加了一个L2的loss。相当于做了一个L2的正则化处理。
为了理解L2正则化我们先把L2正则化的公式列下来：
$$ J(w,b) = \frac{1}{m}\sum^{m}_{i=1}L(w,b) + \frac{\gamma}{2m}||w||^{2}_{2}$$
其实也就是给成本函数添加一个正则项。
这样一来，调整参数的时候，公式就变成了这样：
$$ w = (1-\alpha\frac{\gamma}{m})w - \alpha\frac{\partial J}{\partial w}$$
对正则化的理解：首先，正则化是为了减少模型的过拟合，一般我们可以通过减少特征或者惩罚特征来达到这一个目的，因为通过惩罚特征，不让神经网络过度依赖某一些特征可以提高神经网络的泛化性，而正则化就是帮助我们来惩罚特征的。在上面的第一个式子我们可以 看到L2正则化就是在损失函数后面添加了一个权重损失项，也可以这么理解，为了使用某个权重，我们就需要付出某些loss的代价，除非这个特征非常有效，否则就会被loss上的增加覆盖效果。这样我们就可以筛选出最有效的特征，减少特征权重过拟合。

因此通过对上面公式的理解，我们有了下面的代码，我们首先，先计算出每一个权重的l2正则化的损失
```python
    tf.nn.l2_loss(var)
```
同时，为了我们可以控制L2正则化的程度，我们又给得到的L2正则化损失添加了一个权重，即下面的代码
```python
    tf.multiply(tf.nn.l2_loss(var), wl, name = 'weight_loss')
```
这就是最终的weight_loss,而通过上面的公式我们可以看到，最终的loss是要添加到损失函数上面的，因此我们这里先把我们的weight_loss先保存起来，到计算损失函数的时候再进行使用，下面的代码就是保存我们weight_loss的代码
```python 
   tf.add_to_collection('losses', weight_loss)
```
因为我们并不是对 我们的权重做的操作，因此只需要原班不动的返回我们的权重即可
```python
    return var
```

In [3]:
def variable_with_weight_loss(shape, stddev, wl):
    var = tf.Variable(tf.truncated_normal(shape, stddev=stddev))
    if wl is not None:
        weight_loss = tf.multiply(tf.nn.l2_loss(var), wl, name = 'weight_loss')
        tf.add_to_collection('losses', weight_loss)
    return var

使用distorted_inputs产生训练数据，这里返回是封装好的tensor，每次执行都会生成一个batch_size数量的样本，同时，这里使用了Data Augmentation（数据增强），数据增强包括随机的顺平翻转，随机剪切一块24\*24大小的图片，设置随机的亮度和对比度，以及对数据进行标准化。

另外，distorted_inputs使用了16个独立的线程来加速任务，函数内部会产生线程池，在需要使用时会通过TensorFlow queue进行调度

In [4]:
images_train, labels_train = cifar10_input.distorted_inputs(data_dir=data_dir, batch_size=batch_size)

Filling queue with 20000 CIFAR images before starting to train. This will take a few minutes.


使用cifar10_inputs.inputs函数生成测试数据，这里不需要对图片进行翻转或修改亮度，对比度，不过需要裁剪图片正中间的24x24大小的区块，并对数据进行标准化处理

In [5]:
images_test, labels_test = cifar10_input.inputs(eval_data=True, data_dir = data_dir, batch_size = batch_size)

这里定义两个占位符，不再多说，一个是训练集X，一个是输出标签Y

In [6]:
image_holder = tf.placeholder(tf.float32, [batch_size, 24, 24, 3])
label_holder = tf.placeholder(tf.int32, [batch_size])

定义第一个卷积层，这个卷积层中

首先我们定义权重参数W（weight1）--卷积核

然后让卷积核和输入进行卷积操作

然后应用激活函数，这里使用的是ReLU函数，这里的bias_add是add的一种特例，使用情况是必须保证bias和input的最后一维的维度相同，这里，我们的卷积处理完成之后，数据的维度是[batch_size, w, h, 64]，我们的bias维度是[64]，因此，维度相同

之后使用一个尺寸为3\*3，步长为2\*2的最大池化层处理数据。这里的最大池化的尺寸和步长不一致，这样可以增加数据的丰富性。

最后使用**tf.nn.lrn**函数，即LRN对结果进行处理，Alex在论文中解释LRN层模仿了生物神经系统的侧抑制机制，对局部神经元的活动创建竞争环境，使得其中响应比较大的值变得相对更大，并抑制其他反馈较小的神经元，增强了模型的泛化能力。Alex在ImageNet数据集上的实验表明，使用LRN后CNN在Top1的错误率可以降低1.4%，因此在其经典的AlexNet中使用了LRN层。LRN对ReLU这种没有上限边界的激活函数会比较有用，因为它会从附近的多个卷积核的相应中挑选比较大的反馈，但不适合sigmoid这种有固定边界并且能抑制过大值的激活函数。

In [7]:
weight1 = variable_with_weight_loss(shape=[5,5,3,64], stddev=5e-2, wl=0.0)
kernel = tf.nn.conv2d(image_holder, weight1, [1,1,1,1], padding="SAME")
bias1 = tf.Variable(tf.constant(0.0, shape=[64]))
conv1 = tf.nn.relu(tf.nn.bias_add(kernel, bias1))
pool1 = tf.nn.max_pool(conv1, ksize=[1,3,3,1], strides=[1,2,2,1], padding="SAME")
norm1 = tf.nn.lrn(pool1, 4, bias = 1.0, alpha=0.001/9.0, beta=0.75)

创建第二个卷积层，这里的初始化参数以及卷积操作就不多说了。

这里第二个卷积层和上面的第一个卷积层的不同的地方是，这里我们**调换了最大池化层和LRN层的顺序**

In [8]:
weight2 = variable_with_weight_loss(shape=[5,5,64,64], stddev=5e-2, wl=0.0)
kernel2 = tf.nn.conv2d(norm1, weight2, strides=[1,1,1,1], padding='SAME')
bias2 = tf.Variable(tf.constant(0.1, shape=[64]))
conv2 = tf.nn.relu(tf.nn.bias_add(kernel2, bias2))
norm2 = tf.nn.lrn(conv2, 4, bias = 1.0, alpha = 0.001/9.0, beta = 0.75)
pool2 = tf.nn.max_pool(norm2, ksize=[1,3,3,1], strides=[1,2,2,1], padding='SAME')

接下来使用全连接层，同样的模板操作，将上一层的输出reshape成一维数据。
这里有一个技巧，我们不需要再进行计算最终的卷积层输出是什么了，而是直接使用[batch_size, -1]来进行reshape，这样就减少了reshape的计算维度的步骤了，之后我们可以直接使用方法**reshape.get_shape()[1].value**来获取卷积层的最终输出维度。

In [9]:
reshape = tf.reshape(pool2, [batch_size, -1])
dim = reshape.get_shape()[1].value
weight3 = variable_with_weight_loss(shape=[dim, 384], stddev=0.04, wl = 0.004)
bias3 = tf.Variable(tf.constant(0.1, shape=[384]))
local3 = tf.nn.relu(tf.matmul(reshape, weight3) + bias3)

第二个全连接层，不用赘述

In [10]:
weight4 = variable_with_weight_loss(shape=[384, 192], stddev=0.04, wl=0.004)
bias4 = tf.Variable(tf.constant(0.1, shape=[192]))
local4 = tf.nn.relu(tf.matmul(local3, weight4) + bias4)

第三个全连接层，需要注意的是，这里不像之前那样使用softmax输出最后结果，这是因为我们把softmax的操作放在了计算loss的部分，我们不需要对inference的输出进行softmax处理就可以获得最终分类结果。（直接比较inference输出的各类的数值大小即可），计算softmax只要是为了计算loss，因此，softmax操作整合到后面是比较合适的。

**这里说明一下，首先，无论是否使用softmax激活函数，结果是输出哪一个类别并不会改变，softmax的作用就是把输出结果归一化，并不会影响各个输出值的大小，这是作者不使用softmax的原因。另外，之所以使用softmax是因为后面计算损失函数的方便，因此，作者把softmax的计算归到了loss里面。因为我们在之前学习多类别训练的时候使用的就是softmax函数，损失函数也是在这个基础上计算出来的，其实无论最后一层是否使用softmax函数，不影响最终的输出，但是loss函数的计算得运用softmax函数。只不过如果最后一层不适用softmax函数的话，那么，每一个节点的输出的最终结果也就不会是概率了。总之，这也是换了一种方式的实现吧。**

In [11]:
weight5 = variable_with_weight_loss(shape=[192, 10], stddev=1/192.0, wl = 0.)
bias5 = tf.Variable(tf.constant(0.0, shape=[10]))
logits = tf.add(tf.matmul(local4, weight5), bias5)

计算网络的损失函数，这里我们把softmax和cross_entropy_loss的计算合在了一起，即函数**tf.nn.sparse_softmax_cross_entropy_with_logits**, *也就是说，如果在多分类网络模型中，如果最后一层的输出我们不使用softmax激活函数二把softmax归并到损失函数里面进行计算的话，可以直接使用上面的函数。*

在计算出没有使用正则化的损失函数的值之后，我们把这个损失值也添加到整体的loss的collection中，最后使用tf.add_n将整体losses的collection中的全部loss进行求和，这样也就得到了含有L2正则项的损失函数。

In [12]:
def loss(logits, labels):
    labels = tf.cast(labels, tf.int64)
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits,
                                                                  labels=labels,
                                                                  name = 'cross_entropy_per_example')
    cross_entropy_mean = tf.reduce_mean(cross_entropy, name="cross_entropy")
    tf.add_to_collection('losses', cross_entropy_mean)
    return tf.add_n(tf.get_collection('losses'), name='total_loss')

我们将模型的输出logits和数据的标签来计算损失。

In [13]:
loss = loss(logits, label_holder)

设置优化器，如果max_steps比较大，那么推荐使用学习速率衰减的SGD进行训练，这样训练过程中能达到的准确率峰值会比较高，大致接近86%,

In [14]:
train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

使用tf.nn.in_top_k函数求输出结果中top_k的准确率，默认使用top_1,也就是输出分数最高的那一类的准确率。

In [15]:
top_k_op = tf.nn.in_top_k(logits, label_holder, 1)

In [16]:
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

启动前面提到的图片数据增强的线程队列，这里一共使用了16个线程来进行训练加速，如果这里不启动线程，那么后续的inference及训练的操作都是无法开始的。

In [17]:
tf.train.start_queue_runners()

[<Thread(Thread-6, started daemon 14052)>,
 <Thread(Thread-7, started daemon 17100)>,
 <Thread(Thread-8, started daemon 7004)>,
 <Thread(Thread-9, started daemon 16208)>,
 <Thread(Thread-10, started daemon 3356)>,
 <Thread(Thread-11, started daemon 12008)>,
 <Thread(Thread-12, started daemon 13052)>,
 <Thread(Thread-13, started daemon 13432)>,
 <Thread(Thread-14, started daemon 10996)>,
 <Thread(Thread-15, started daemon 11484)>,
 <Thread(Thread-16, started daemon 8196)>,
 <Thread(Thread-17, started daemon 20232)>,
 <Thread(Thread-18, started daemon 12620)>,
 <Thread(Thread-19, started daemon 16748)>,
 <Thread(Thread-20, started daemon 13836)>,
 <Thread(Thread-21, started daemon 21340)>,
 <Thread(Thread-22, started daemon 5748)>,
 <Thread(Thread-23, started daemon 23304)>,
 <Thread(Thread-24, started daemon 1952)>,
 <Thread(Thread-25, started daemon 2308)>,
 <Thread(Thread-26, started daemon 19004)>,
 <Thread(Thread-27, started daemon 12188)>,
 <Thread(Thread-28, started daemon 2280)>,

进行训练

In [18]:
for step in range(max_step):
    start_time = time.time()
    image_batch, label_batch = sess.run([images_train, labels_train])
    _,loss_value = sess.run([train_op, loss],
                           feed_dict={image_holder:image_batch, label_holder:label_batch})
    duration = time.time() - start_time
    if step % 10 == 0:
        example_per_sec = batch_size/duration
        sec_per_batch = float(duration)
        format_str = ('step %d, loss=%.2f (%.1f example/sec; %.3f sec/batch)')
        print(format_str %(step, loss_value, example_per_sec, sec_per_batch))

step 0, loss=4.68 (82.5 example/sec; 1.552 sec/batch)
step 10, loss=3.70 (113.5 example/sec; 1.127 sec/batch)
step 20, loss=3.03 (111.4 example/sec; 1.149 sec/batch)
step 30, loss=2.68 (107.3 example/sec; 1.193 sec/batch)
step 40, loss=2.49 (93.7 example/sec; 1.366 sec/batch)
step 50, loss=2.28 (119.5 example/sec; 1.071 sec/batch)
step 60, loss=2.18 (112.7 example/sec; 1.136 sec/batch)
step 70, loss=2.08 (111.5 example/sec; 1.148 sec/batch)
step 80, loss=2.11 (112.0 example/sec; 1.143 sec/batch)
step 90, loss=2.04 (107.8 example/sec; 1.187 sec/batch)
step 100, loss=2.01 (113.4 example/sec; 1.129 sec/batch)
step 110, loss=1.98 (102.9 example/sec; 1.244 sec/batch)
step 120, loss=1.92 (107.2 example/sec; 1.194 sec/batch)
step 130, loss=1.88 (111.3 example/sec; 1.150 sec/batch)
step 140, loss=1.79 (99.8 example/sec; 1.283 sec/batch)
step 150, loss=1.86 (108.8 example/sec; 1.177 sec/batch)
step 160, loss=1.91 (115.5 example/sec; 1.108 sec/batch)
step 170, loss=1.80 (113.3 example/sec; 1.130

step 1440, loss=1.27 (117.4 example/sec; 1.090 sec/batch)
step 1450, loss=1.32 (112.6 example/sec; 1.136 sec/batch)
step 1460, loss=1.18 (107.3 example/sec; 1.193 sec/batch)
step 1470, loss=1.29 (117.9 example/sec; 1.086 sec/batch)
step 1480, loss=1.14 (97.8 example/sec; 1.308 sec/batch)
step 1490, loss=1.32 (102.0 example/sec; 1.255 sec/batch)
step 1500, loss=1.21 (113.8 example/sec; 1.124 sec/batch)
step 1510, loss=1.16 (116.8 example/sec; 1.096 sec/batch)
step 1520, loss=1.11 (114.7 example/sec; 1.116 sec/batch)
step 1530, loss=1.15 (105.2 example/sec; 1.216 sec/batch)
step 1540, loss=1.22 (114.4 example/sec; 1.119 sec/batch)
step 1550, loss=1.23 (94.1 example/sec; 1.360 sec/batch)
step 1560, loss=1.22 (93.8 example/sec; 1.364 sec/batch)
step 1570, loss=1.38 (115.8 example/sec; 1.105 sec/batch)
step 1580, loss=1.16 (107.0 example/sec; 1.196 sec/batch)
step 1590, loss=1.21 (112.3 example/sec; 1.139 sec/batch)
step 1600, loss=1.11 (115.3 example/sec; 1.110 sec/batch)
step 1610, loss=1

step 2860, loss=1.13 (101.4 example/sec; 1.262 sec/batch)
step 2870, loss=1.19 (100.8 example/sec; 1.269 sec/batch)
step 2880, loss=1.02 (102.5 example/sec; 1.248 sec/batch)
step 2890, loss=1.11 (99.7 example/sec; 1.284 sec/batch)
step 2900, loss=1.06 (101.1 example/sec; 1.266 sec/batch)
step 2910, loss=1.00 (106.0 example/sec; 1.207 sec/batch)
step 2920, loss=1.08 (109.2 example/sec; 1.172 sec/batch)
step 2930, loss=1.10 (106.2 example/sec; 1.205 sec/batch)
step 2940, loss=1.05 (97.2 example/sec; 1.317 sec/batch)
step 2950, loss=1.13 (99.4 example/sec; 1.287 sec/batch)
step 2960, loss=1.24 (114.0 example/sec; 1.123 sec/batch)
step 2970, loss=1.10 (107.0 example/sec; 1.196 sec/batch)
step 2980, loss=1.16 (80.9 example/sec; 1.583 sec/batch)
step 2990, loss=1.09 (113.5 example/sec; 1.128 sec/batch)


计算正确率

In [19]:
num_examples = 10000
import math
num_iter = int(math.ceil(num_examples/batch_size))
ture_count = 0
total_sample_count = num_iter * batch_size
step = 0
while step < num_iter:
    image_batch, label_batch = sess.run([images_test, labels_test])
    predictions = sess.run([top_k_op], feed_dict={image_holder:image_batch, label_holder:label_batch})
    ture_count += np.sum(predictions)
    step += 1

In [21]:
precision = ture_count / total_sample_count
print('precision @ 1 = %.3f' % precision)

precision @ 1 = 0.725


另外，数据增强在训练中作用很大，它可以给单幅图增加多个副本，大大增加了样本量，从而提高图片的利用率，防止对某一张图片结构的学习过拟合。图片本身冗余信息比较多，因此，可以制造不同的噪声并让图片依然可以被识别出来。如果神经网络可以克服这些噪声，那么它的泛化性一定很好。下图是数据量和不同的模型准确率的关系。![img/1.png](img/1.png)

从上面和之前的例子可以看到，卷积层一般需要和一个池化层链接，卷积加池化已经是图像识别的一个标准组件了。后面的全连接层的目的是输出分类结果，前面卷积层是特征提取工作，直到最后全连接层才开始对特征进行组合匹配并进行分类。

卷积层的训练相比于全连接层更复杂，尽管全连接层参数占全部参数的90%多。训练全连接层基本是进行一些矩阵乘法运算，而卷积层的训练基本依赖于cuDNN的实现。算法相对复杂，有些方法还会涉及傅里叶变换。